In [1]:
import gensim.models.keyedvectors as word2vec
import time
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
import os
import matplotlib.pyplot as plt
from pylab import *
import scipy
from sklearn.linear_model import LogisticRegression
import warnings
from scipy.stats import pearsonr
warnings.filterwarnings('ignore')

The first block here takes all names encoded and used to train the initial sound symbolism model (from file human_names_coded.csv). A logistic regression is fit using the feature vectors from the sound symbolism study.

In [2]:
df_human = pd.read_csv('../Brand-Name-Gender-Prediction/human_names_coded.csv')
df_human = df_human.drop(["dataID"], axis=1)
df_human = df_human.drop(["Name"], axis=1)
x = df_human.drop(['Gender', 'is_unique'], axis=1).as_matrix()[:, 1:]
y = df_human[['Gender']].as_matrix().ravel()
model_LR = LogisticRegression()
model_LR.fit(x, y)



LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

The block below predicts the male/female alignment likelihoods for each brand from the preprocessed Kantar dataset with no duplicates (n=2308). The first index of the logistic regression output is likelihood for being male, the second index is the likelihood for being female. The likelihood for being female is added to "maps" to compare with likelihood for being female from previous work we conducted on gender alignment along he-she axis.

In [ ]:
df_brands = pd.read_csv('brand_names_coded_no_duplicates.csv')
df_brands
maps = {}
for i in range(len(df_brands.index)):
    feature_vec = df_brands.iloc[i][4:].as_matrix().ravel().reshape((1, -1))
    x = model_LR.predict_proba(feature_vec)
    print(df_brands.iloc[i][2], x)
    maps[df_brands.iloc[i][2]] = x[0][1]

For all brands in he_she_alignment_just_brands.csv (trained by Word2Vec on Google News corpus), we find the gender alignment by reading from the csv and append it to list he_she. The logistic regression value is added to the corresponding index in list log_reg. 

In [4]:
df_align = pd.read_csv('he_she_alignment_just_brands.csv')
brands = list(maps.keys())
log_reg = []
he_she = []
for index, row in df_align.iterrows():
    brand = row['name']
    if brand in brands:
        log_reg.append(maps[brand])
        he_she.append(row['alignment'])
        


Finding the Pearson correlation here (data seems to have normal distribution so assumptions are satisfied).

In [10]:
x = pearsonr(he_she, log_reg)
vals = list(map(np.round, x, [3, 10]))
print("Correlation coefficient of {}, p-value of {}".format(vals[0], vals[1]))

Correlation coefficient of 0.125, p-value of 1.5226e-06
